# RAG 

#### Libraries

In [49]:
# processing:
import os
import re
import pandas as pd
from dotenv import load_dotenv

# neo4j:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# llm:
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retrieval:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

#### .env

In [50]:
load_dotenv()

True

In [51]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
LLM_SCOPE = os.getenv('SCOPE')
LLM_AUTH = os.getenv('AUTH_DATA')

# telegram:
BOT_TOKEN = os.getenv('BOT_TOKEN')

#### Reference

In [52]:
reference = pd.read_excel('../docs/reference.xlsx')

## Setting Up LLM | GigaChat


[GigaChat](https://developers.sber.ru/docs/ru/gigachat/overview)

In [53]:
llm = GigaChat(credentials = LLM_AUTH, 
               temperature = 0.1,
               n = 1, 
               model = "GigaChat-Plus", # 32k context window
               repetition_penalty = 1.0,
               verify_ssl_certs = False)

In [54]:
parser = StrOutputParser()

In [55]:
embeddings = GigaChatEmbeddings(credentials = LLM_AUTH, verify_ssl_certs = False)

## Prompt Engineering

#### Prompt Template

In [95]:
template = """
            Задача: Анализировать заданный запрос и предоставлять детализированные ответы, опираясь на документы, правила и требования.

            Инструкции:

            1. При ответе на вопросы, особенно те, которые связаны с правилами или нормативными документами, активно ссылайтесь на номера пунктов, статей и разделов документов.
            2. Если требуемая информация отсутствует или вы не можете обнаружить необходимые пункты в документах, используйте фразу "Информация не найдена".
            3. Ваши ответы должны быть максимально точными и содержать не только ссылки на документы, но и объяснения их применения к данному запросу.

            Вопрос: {question}
            Контекст: {context}
            
            Ответ: Предоставьте ваш ответ, опираясь на указанные выше указания. создай окончательный ответ с ссылками ("SOURCES"). 
            """

prompt = ChatPromptTemplate.from_template(template)

#### Step Back Prompting

In [57]:
def step_back_prompt(model, parser, text):
    
    chain = model | parser

    template = f"""
                Задача: Анализировать представленные данные и вывести из них высокоуровневые концепции и основные принципы.

                Инструкции:
                Прежде чем приступить к абстракции, важно:
                1. Определить и выделить ключевые детали и специфику представленного материала.
                2. Анализировать связи между деталями для понимания более глубоких закономерностей и взаимосвязей.
                3. Структурировать полученные данные, выделяя общие элементы и паттерны.
                4. Формулировать высокоуровневые концепции и принципы, опираясь на проанализированные данные.

                Текст: \"{text}\"

                Проанализируйте и сформулируйте общие концепции и основные принципы на основе представленных деталей в виде развернутых вопросов.
                """

    model_response = chain.invoke(template)
    
    return model_response

#### Question Extraction

In [58]:
def extract_question(model, parser, text):
    
    chain = model | parser

    template = f"""
                            Мне нужно твоё содействие в анализе следующего делового письма.
                            Извлеки из него всю важную информацию для системы RAG. 
                            Нужны вопросы, ключевые параметры, и основные темы обсуждения. Вот текст письма:

                            Текст: \"{text}\"

                            Разбери письмо на следующие компоненты:

                            1. Все поставленные вопросы.
                            2. Перечень ключевых параметров, упомянутых в тексте.
                            3. Основные темы, которые обсуждаются.

                            Используй структурированный подход, чтобы я мог легко использовать эти данные для запросов в системе RAG.
                            Без дополнительных комментариев.
                            """

    model_response = chain.invoke(template)
    
    return model_response

## Setting Up Database | Neo4j

[Neo4j](https://workspace-preview.neo4j.io)

In [59]:
neo4j_vector = Neo4jVector.from_existing_index(embeddings,
                                               url=NEO4J_URI,
                                               username=NEO4J_USERNAME,
                                               password=NEO4J_PASSWORD,
                                               index_name="vector",  # vector by default
                                               node_label="Chunk"  # Chunk by default
                                               )

## LLM Retrival Chain

[GigaChain](https://github.com/ai-forever/gigachain)

#### Chain

In [104]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
                                                    chain_type = "stuff", # "stuff", "map_rerank", "refine"
                                                    retriever = neo4j_vector.as_retriever(search_kwargs={"k": 5, 'lambda_mult': 0.25}),
                                                    return_source_documents = True,
                                                    reduce_k_below_max_tokens=False,
                                                    max_tokens_limit=32000,
                                                    chain_type_kwargs = {
                                                                        "verbose": True,
                                                                        "prompt": prompt, # step_back_prompt
                                                                        "document_variable_name": "context",
                                                                        "memory": ConversationBufferMemory(
                                                                            memory_key='history',
                                                                            input_key='question'),
                                                                        }
                                                    )

## Test

##### Reference Letters + Answers

In [61]:
letter = 7 # from the reference.xlsx num + 2

inquiry = str(reference['letter'][letter].replace('\n', ' '))
refer = str(reference['answer '][letter]).replace('\n', ' ')

abstraction = step_back_prompt(llm, parser, inquiry)
question = extract_question(llm, parser, inquiry)

##### Chain

In [106]:
# print(f'Inquiry: {inquiry}\n')
# print(f'Reference: {refer}\n')

# print(f'Question: {question}\n')

chain.invoke(
            {"question": question},
            return_only_outputs = False
            )



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
            Задача: Анализировать заданный запрос и предоставлять детализированные ответы, опираясь на документы, правила и требования.

            Инструкции:

            1. При ответе на вопросы, особенно те, которые связаны с правилами или нормативными документами, активно ссылайтесь на номера пунктов, статей и разделов документов.
            2. Если требуемая информация отсутствует или вы не можете обнаружить необходимые пункты в документах, используйте фразу "Информация не найдена".
            3. Ваши ответы должны быть максимально точными и содержать не только ссылки на документы, но и объяснения их применения к данному запросу.

            Вопрос: 1. Вопросы:
- Нужно ли делать корректировку в ГПЗ при уменьшении НМЦ закупки?
- Нужна ли корректировка в ГПЗ при уменьшении потребности в чел./часах на выполнение работ?

2. Ключевые параметры:
- НМЦ закупки
- Потребно


> Finished chain.

> Finished chain.


{'question': '1. Вопросы:\n- Нужно ли делать корректировку в ГПЗ при уменьшении НМЦ закупки?\n- Нужна ли корректировка в ГПЗ при уменьшении потребности в чел./часах на выполнение работ?\n\n2. Ключевые параметры:\n- НМЦ закупки\n- Потребность в чел./часах на выполнение работ\n- Стоимость договора\n\n3. Основные темы обсуждения:\n- Корректировка в ГПЗ при уменьшении НМЦ закупки\n- Корректировка в ГПЗ при уменьшении потребности в чел./часах на выполнение работ',
 'answer': 'Для анализа вашего запроса, я обращусь к документам, правилам и требованиям, которые были предоставлены.\n\n1. Вопросы:\n- Нужно ли делать корректировку в ГПЗ при уменьшении НМЦ закупки?\n- Нужна ли корректировка в ГПЗ при уменьшении потребности в чел./часах на выполнение работ?\n\nОтвет:\n- Согласно документам, корректировка в ГПЗ должна осуществляться в случае изменения предмета закупки, объема закупаемых товаров, способа закупки, формы закупки, Организатора/Координатора закупки, разделения на лоты/процедуры, объедин

#### Sources

In [63]:
docs_with_score_question = neo4j_vector.similarity_search_with_score(question, k = 5)

for doc, score in docs_with_score_question:
    print("-" * 100)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Score:  0.9527992010116577
5.3.4. Корректировка записей ГПЗ должна осуществляться в следующих случаях: 1) изменение предмета закупки / предмета договора (кроме случаев отмены потребности в первоначальной закупке); 2) изменение объема закупаемых товаров (работ, услуг); 3) изменение месяца инициации закупки на более поздний месяц в пределах года планирования;  4) изменение способа закупки; 5) изменение формы закупки (Изменения с бумажной на электронную форму проведения закупок и наоборот. Изменения с открытой на закрытую форму проведения закупок и наоборот). 6) изменение Организатора / Координатора закупки; 7) разделение на лоты / процедуры, объединение в один лот / процедуру; 8) изменение кода услуги; 9) изменение категории закупки;
----------------------------------------------------------------------------------------------------
--------------------------------------------------------

In [64]:
docs_with_score_question

[(Document(page_content='5.3.4. Корректировка записей ГПЗ должна осуществляться в следующих случаях: 1) изменение предмета закупки / предмета договора (кроме случаев отмены потребности в первоначальной закупке); 2) изменение объема закупаемых товаров (работ, услуг); 3) изменение месяца инициации закупки на более поздний месяц в пределах года планирования;  4) изменение способа закупки; 5) изменение формы закупки (Изменения с бумажной на электронную форму проведения закупок и наоборот. Изменения с открытой на закрытую форму проведения закупок и наоборот). 6) изменение Организатора / Координатора закупки; 7) разделение на лоты / процедуры, объединение в один лот / процедуру; 8) изменение кода услуги; 9) изменение категории закупки;', metadata={'source': 'ГПЗ/5. ПОРЯДОК ВЫПОЛНЕНИЯ ПРОЦЕССА «ФОРМИРОВАНИЕ ГОДОВОГО ПЛАНА ЗАКУПОК»', 'bullet': '5.3.4'}),
  0.9527992010116577),
 (Document(page_content='[1]. консолидированный ГПЗ: свод данных утвержденных ГПЗ всех Обществ Компания 1 и ПАО\xa0«Ко

##### Step Back Chain

In [65]:
print(f'Inquiry: {inquiry}\n')
print(f'Reference: {refer}\n')

print(f'Abstraction: {abstraction}\n')

chain.invoke(
            {"question": abstraction},
            return_only_outputs = True,
            )

Inquiry: Уважаемы коллеги, добрый день! Если в процессе инициации закупки возникла необходимость в уменьшении НМЦ закупки, то нужно ли делать корректировку в ГПЗ? Например, в процессе согласования договора с ВЗЛ, у заказчика немного уменьшилась потребность в чел./часах на выполнение определённых работ, следовательно, снизилась стоимость договора. Состав работ и команда специалистов при этом не изменились. Нужна ли корректировка в этом случае?  С уважением, [Ваше Имя] [Ваша Должность] [Ваша Компания]

Reference: Да, корректировка в данном случае нужна, так как в вашем лучае меняется объем закупаемых товаров (работ, услуг) согласовно пункту 5.4.3 «Формирование ГПЗ».  Более подробно о требованиях и условиях корректировки Годового плана закупок можно прочитать в «Формирование ГПЗ».

Abstraction: На основе представленных данных можно сформулировать следующие общие концепции и основные принципы:

1. При инициации закупки может возникнуть необходимость в уменьшении НМЦ закупки.
2. Если потреб

{'answer': 'На основе представленных данных можно сформулировать следующие общие концепции и основные принципы:\n\n1. При инициации закупки может возникнуть необходимость в уменьшении НМЦ закупки.\n2. Если потребность в чел./часах на выполнение определенных работ уменьшилась, это может привести к снижению стоимости договора.\n3. Состав работ и команда специалистов могут остаться неизменными при корректировке в ГПЗ.\n4. Корректировка в ГПЗ может быть необходима для отражения изменений в потребности заказчика и стоимости договора.\n5. Важно учитывать все детали и специфику при анализе и формулировании общих концепций и основных принципов.\n\nИнформация не найдена.',
 'sources': '',
 'source_documents': [Document(page_content='5.3.4. Корректировка записей ГПЗ должна осуществляться в следующих случаях: 1) изменение предмета закупки / предмета договора (кроме случаев отмены потребности в первоначальной закупке); 2) изменение объема закупаемых товаров (работ, услуг); 3) изменение месяца иниц

In [66]:
docs_with_score_abstraction = neo4j_vector.similarity_search_with_score(abstraction, k = 5)

for doc, score in docs_with_score_abstraction:
    print("-" * 100)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Score:  0.9550783038139343
5.3.4. Корректировка записей ГПЗ должна осуществляться в следующих случаях: 1) изменение предмета закупки / предмета договора (кроме случаев отмены потребности в первоначальной закупке); 2) изменение объема закупаемых товаров (работ, услуг); 3) изменение месяца инициации закупки на более поздний месяц в пределах года планирования;  4) изменение способа закупки; 5) изменение формы закупки (Изменения с бумажной на электронную форму проведения закупок и наоборот. Изменения с открытой на закрытую форму проведения закупок и наоборот). 6) изменение Организатора / Координатора закупки; 7) разделение на лоты / процедуры, объединение в один лот / процедуру; 8) изменение кода услуги; 9) изменение категории закупки;
----------------------------------------------------------------------------------------------------
--------------------------------------------------------

In [67]:
docs_with_score_abstraction

[(Document(page_content='5.3.4. Корректировка записей ГПЗ должна осуществляться в следующих случаях: 1) изменение предмета закупки / предмета договора (кроме случаев отмены потребности в первоначальной закупке); 2) изменение объема закупаемых товаров (работ, услуг); 3) изменение месяца инициации закупки на более поздний месяц в пределах года планирования;  4) изменение способа закупки; 5) изменение формы закупки (Изменения с бумажной на электронную форму проведения закупок и наоборот. Изменения с открытой на закрытую форму проведения закупок и наоборот). 6) изменение Организатора / Координатора закупки; 7) разделение на лоты / процедуры, объединение в один лот / процедуру; 8) изменение кода услуги; 9) изменение категории закупки;', metadata={'source': 'ГПЗ/5. ПОРЯДОК ВЫПОЛНЕНИЯ ПРОЦЕССА «ФОРМИРОВАНИЕ ГОДОВОГО ПЛАНА ЗАКУПОК»', 'bullet': '5.3.4'}),
  0.9550783038139343),
 (Document(page_content='[1]. консолидированный ГПЗ: свод данных утвержденных ГПЗ всех Обществ Компания 1 и ПАО\xa0«Ко

## Response 

##### Temporary solution:

In [68]:
context = [doc[0].page_content for doc in docs_with_score_abstraction + docs_with_score_question]

In [69]:
context

['5.3.4. Корректировка записей ГПЗ должна осуществляться в следующих случаях: 1) изменение предмета закупки / предмета договора (кроме случаев отмены потребности в первоначальной закупке); 2) изменение объема закупаемых товаров (работ, услуг); 3) изменение месяца инициации закупки на более поздний месяц в пределах года планирования;  4) изменение способа закупки; 5) изменение формы закупки (Изменения с бумажной на электронную форму проведения закупок и наоборот. Изменения с открытой на закрытую форму проведения закупок и наоборот). 6) изменение Организатора / Координатора закупки; 7) разделение на лоты / процедуры, объединение в один лот / процедуру; 8) изменение кода услуги; 9) изменение категории закупки;',
 '[1]. консолидированный ГПЗ: свод данных утвержденных ГПЗ всех Обществ Компания 1 и ПАО\xa0«Компания 1» на календарный год. Координатор: подразделение ПАО\xa0«Компания 1» или подразделение Общества Компания 1, ответственное за предоставление пакета документов утверждающему лицу/у

In [70]:
template = f"""
            Задача: Анализировать заданный запрос и предоставлять детализированные ответы, опираясь на документы, правила и требования.

            Инструкции:

            1. При ответе на вопросы, особенно те, которые связаны с правилами или нормативными документами, активно ссылайтесь на номера пунктов, статей и разделов документов.
            2. Если требуемая информация отсутствует или вы не можете обнаружить необходимые пункты в документах, используйте фразу "Информация не найдена".
            3. Ваши ответы должны быть максимально точными и содержать не только ссылки на документы, но и объяснения их применения к данному запросу.

            Вопрос: {question}
            Контекст: {context}
            
            Ответ: Предоставьте ваш ответ, опираясь на указанные выше указания.
            """


In [71]:
chains = llm | parser
chains.invoke(template)

'Для ответа на ваши вопросы, я проанализировал предоставленную информацию и пришел к следующим выводам:\n\n1. Вопросы:\n- Нужно ли делать корректировку в ГПЗ при уменьшении НМЦ закупки?\n- Нужна ли корректировка в ГПЗ при уменьшении потребности в чел./часах на выполнение работ?\n\nОтвет:\n- Да, корректировка в ГПЗ должна быть сделана при уменьшении НМЦ закупки. Это связано с тем, что изменение НМЦ может повлиять на объем закупаемых товаров (работ, услуг), что требует корректировки записей ГПЗ.\n- Да, корректировка в ГПЗ должна быть сделана при уменьшении потребности в чел./часах на выполнение работ. Это связано с тем, что изменение потребности может повлиять на объем закупаемых товаров (работ, услуг), что требует корректировки записей ГПЗ.\n\nИнформация не найдена:\n- Нет информации о необходимости корректировки в ГПЗ при уменьшении потребности в чел./часах на выполнение работ.\n- Нет информации о необходимости корректировки в ГПЗ при уменьшении потребности в чел./часах на выполнение р

## Telegram

In [72]:
import telebot
from time import sleep

In [73]:
user_conversations = {}
bot = telebot.TeleBot(BOT_TOKEN)

In [74]:
question = extract_question(llm, parser, inquiry)

chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
                                                    chain_type = "stuff", # "stuff", "map_rerank", "refine"
                                                    retriever = neo4j_vector.as_retriever(search_kwargs={"k": 5}),
                                                    return_source_documents = False,
                                                    reduce_k_below_max_tokens=False,
                                                    max_tokens_limit=32000,
                                                    chain_type_kwargs = {
                                                                        "verbose": False,
                                                                        "prompt": prompt, # step_back_prompt
                                                                        "document_variable_name": "context",
                                                                        "memory": ConversationBufferMemory(
                                                                            memory_key='history',
                                                                            input_key='question'),
                                                                        }
                                                    )

@bot.message_handler(content_types=['audio', 'video', 'document', 'photo', 'sticker', 'voice', 'location', 'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')

@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id
    question = extract_question(llm, parser, message.text)

    response = chain.invoke(
            {"question": question},
            return_only_outputs = True
            )["answer"]

    bot.send_message(user_id, response)
    sleep(2)

In [75]:
bot.polling(none_stop=True)

## Basic Queries | Neo4j

[Cypher](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise)

In [ ]:
graph = Neo4jGraph(url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE)

In [ ]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING}
Relationship properties are the following:

The relationships are the following:



In [ ]:
cypher = """
  SHOW VECTOR INDEXES
  """
graph.query(cypher)

[{'id': 5,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 16, 13, 21, 38, 725000000, tzinfo=<UTC>),
  'readCount': 43}]

In [ ]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """
graph.query(cypher)

[{'count(n)': 1030}]

In [ ]:
cypher = """
    MATCH (n:Chunk {bullet: "1.3.1"})
    RETURN n.text AS text
    """
graph.query(cypher)

[{'text': '1.3.1. Планирование закупок ПАО «Компания 1» и Обществ Компания 1, в рамках  которого:'}]

In [ ]:
highest_scored_content = str(neo4j_vector.similarity_search(inquiry, k=3))

match = re.search(r'\d+(\.\d+)+', highest_scored_content)

if match:
    highest_scored_document_number = match.group(0)
    print("Highest scored document number:", highest_scored_document_number)

else:
    print("No matching number found")

if match:
    cypher_query = f"""
                    MATCH (n:Chunk)
                    WHERE n.bullet STARTS WITH '{highest_scored_document_number}'
                    RETURN n.text
                    """

Highest scored document number: 4.5.13


In [ ]:
graph.query(cypher_query)

[{'n.text': '4.5.13. Начальная (максимальная) цена договора (предмета закупки) методом  сопоставимых рыночных цен (анализ рынка) определяется по формуле: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖 где: НМЦ - начальная (максимальная) цена договора (предмета закупки), определяемая методом сопоставимых рыночных цен (анализ рынка);𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете;𝑖 - номер источника ценовой информации; Ц𝑖 - цена единицы товара (работы, услуги), представленная в источнике с номером , скорректированная с учетом коэффициентов𝑖 (индексов), применяемых для пересчета цен товаров (работ, услуг) с учетом различий в характеристиках товаров, коммерческих и (или) финансовых условий поставок товаров (выполнения работ, оказания услуг), определяемых в соответствии с пунктом 4.5.10.'}]